# General Steps
 - Load cleaned data
 - Train-Test split
 - TF-IDF vectorization

In [ ]:
# Core libraries
import pandas as pd
import numpy as np

# Train / test split
from sklearn.model_selection import train_test_split

# Text vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

# Model persistence
import joblib


In [3]:
# Load preprocessed dataset
DATA_PATH = "E:\\Desktop\\02VC_Group_09\\datasets\\SMS_spam_collection\\sms_clean.csv"
df = pd.read_csv(DATA_PATH)

# Encode labels: ham -> 0, spam -> 1
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Features and target
X = df['clean_message']
y = df['label']

print("Dataset size:", df.shape)
print(df.head())


Dataset size: (5572, 2)
   label                                      clean_message
0      0  go jurong point crazi avail bugi n great world...
1      0                              ok lar joke wif u oni
2      1  free entri wkli comp win fa cup final tkt st m...
3      0                u dun say earli hor u c alreadi say
4      0          nah dont think goe usf live around though


In [4]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # preserve class distribution
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])


Training samples: 4457
Testing samples: 1115


In [5]:
# Handle missing values
X_train = X_train.fillna('')
X_test = X_test.fillna('')

print("Missing values in training set:", X_train.isna().sum())
print("Missing values in test set:", X_test.isna().sum())

Missing values in training set: 0
Missing values in test set: 0


In [6]:
# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2),   # unigrams + bigrams
    min_df=2
)

# Fit on training data, transform both sets
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

print("TF-IDF feature shape:", X_train_vec.shape)


TF-IDF feature shape: (4457, 3000)


In [56]:
def predict_sms(message, model, vectorizer, dense=False):
    """
    Predict spam/ham for a single SMS message
    """
    vec = vectorizer.transform([message])
    if dense:
        vec = vec.toarray()
    pred = model.predict(vec)[0]
    return "Spam" if pred == 1 else "Ham"

# 1. Bagging
- Random Forest
- Bagging Classifier + SVM


## 1.1. Random Forest

In [ ]:
# Random Forest model
from sklearn.ensemble import RandomForestClassifier

In [7]:
# Initialize Random Forest classifier
rf_model = RandomForestClassifier(
    n_estimators=200,        # number of trees
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1,               # use all CPU cores
    class_weight="balanced"  # handle class imbalance
)

# Train model
rf_model.fit(X_train_vec, y_train)


RandomForestClassifier(class_weight='balanced', n_estimators=200, n_jobs=-1,
                       random_state=42)

In [8]:
# Predictions
y_pred = rf_model.predict(X_test_vec)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9766816143497757

Classification Report:
              precision    recall  f1-score   support

         Ham       0.97      1.00      0.99       966
        Spam       1.00      0.83      0.90       149

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.95      1115
weighted avg       0.98      0.98      0.98      1115


Confusion Matrix:
[[966   0]
 [ 26 123]]


In [39]:
# Save model and vectorizer
joblib.dump(rf_model, r"E:\Desktop\02VC_Group_09\spamSMS\random_forest_sms.joblib")
joblib.dump(tfidf, r"E:\Desktop\02VC_Group_09\spamSMS\tfidf_vectorizer.joblib")

['E:\\Desktop\\02VC_Group_09\\spamSMS\\tfidf_vectorizer.joblib']

In [24]:
example= "Congratulations! You've won a free prize. Call now!"
example_vec = tfidf.transform([example])
prediction = rf_model.predict(example_vec)
print("Prediction for example message:", "Spam" if prediction[0] == 1 else "Ham")

Prediction for example message: Ham


In [58]:
predict_sms(
    "Urrgent! text me to get your lucky prize iphone 17 promax",
    rf_model,
    tfidf
)

'Ham'

## 1.2. Bagging Classifier + SVM

In [12]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier

In [13]:
# Base SVM model for text data
svm_base = LinearSVC(
    C=1.0,
    class_weight="balanced",
    max_iter=5000,
    random_state=42
)

In [14]:
bagging_svm = BaggingClassifier(
    estimator=svm_base,
    n_estimators=15,        # number of bootstrap SVM models
    max_samples=0.8,        # percentage of samples per estimator
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)

# Train the model
bagging_svm.fit(X_train_vec, y_train)

BaggingClassifier(estimator=LinearSVC(class_weight='balanced', max_iter=5000,
                                      random_state=42),
                  max_samples=0.8, n_estimators=15, n_jobs=-1, random_state=42)

In [15]:
y_pred = bagging_svm.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Ham", "Spam"]))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9829596412556054

Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      0.99      0.99       966
        Spam       0.96      0.91      0.93       149

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115


Confusion Matrix:
[[961   5]
 [ 14 135]]


In [ ]:
joblib.dump(bagging_svm, r"E:\Desktop\02VC_Group_09\spamSMS\bagging_svm_sms.joblib")
# joblib.dump(tfidf, r"E:\Desktop\02VC_Group_09\spamSMS\tfidf_vectorizer.joblib")

['E:\\Desktop\\02VC_Group_09\\spamSMS\\tfidf_vectorizer.joblib']

In [26]:
example_sms = ["Congratulations! You have won a free prize. Call now"]

example_vec = tfidf.transform(example_sms)
prediction = bagging_svm.predict(example_vec)[0]

print("Prediction:", "Spam" if prediction == 1 else "Ham")

Prediction: Spam


## 2. Boosting
- AdaBoost
- Gradient Boosting Classifier

## 2.1. AdaBoost

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [19]:
# Use shallow decision tree (decision stump) as weak learner
base_estimator = DecisionTreeClassifier(
    max_depth=1,
    random_state=42
)

adaboost_model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=200,
    learning_rate=0.5,
    random_state=42
)

# Train model
adaboost_model.fit(X_train_vec, y_train)

C:\Users\n33r9\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1,
                                                    random_state=42),
                   learning_rate=0.5, n_estimators=200, random_state=42)

In [20]:
y_pred = adaboost_model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Ham", "Spam"]))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9748878923766816

Classification Report:
              precision    recall  f1-score   support

         Ham       0.98      0.99      0.99       966
        Spam       0.95      0.86      0.90       149

    accuracy                           0.97      1115
   macro avg       0.96      0.93      0.94      1115
weighted avg       0.97      0.97      0.97      1115


Confusion Matrix:
[[959   7]
 [ 21 128]]


In [27]:
joblib.dump(adaboost_model, r"E:\Desktop\02VC_Group_09\spamSMS\adaboost_sms.joblib")
# joblib.dump(tfidf, r"E:\Desktop\02VC_Group_09\spamSMS\tfidf_vectorizer.joblib")

['E:\\Desktop\\02VC_Group_09\\spamSMS\\adaboost_sms.joblib']

In [28]:
example_sms = ["Congratulations! You have won a free prize. Call now"]  
example_vec = tfidf.transform(example_sms)
prediction = adaboost_model.predict(example_vec)[0]
print("Prediction:", "Spam" if prediction == 1 else "Ham")

Prediction: Spam


## 2.2. Gradient Boosting Classifier

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

In [30]:
gb_model = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

# Train model
gb_model.fit(X_train_vec.toarray(), y_train)

GradientBoostingClassifier(n_estimators=150, random_state=42, subsample=0.8)

In [31]:
y_pred_gb = gb_model.predict(X_test_vec.toarray())

print("Accuracy:", accuracy_score(y_test, y_pred_gb))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb, target_names=["Ham", "Spam"]))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_gb))

Accuracy: 0.9695067264573991

Classification Report:
              precision    recall  f1-score   support

         Ham       0.97      1.00      0.98       966
        Spam       0.98      0.79      0.87       149

    accuracy                           0.97      1115
   macro avg       0.97      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115


Confusion Matrix:
[[963   3]
 [ 31 118]]


In [ ]:
joblib.dump(gb_model, r"E:\Desktop\02VC_Group_09\spamSMS\gradient_boosting_sms.joblib")
# joblib.dump(tfidf, r"E:\Desktop\02VC_Group_09\spamSMS\tfidf_vectorizer.joblib")

['E:\\Desktop\\02VC_Group_09\\spamSMS\\tfidf_vectorizer.joblib']

In [ ]:
new_sms = [
    "Free entry in weekly competition to win cash prize",
    "I'll call you later when I get home"
]
tfidf.fit(X_train)
# TF-IDF transformation using the trained vectorizer
new_sms_vec = tfidf.transform(new_sms).toarray()


# Predict
predictions = gb_model.predict(new_sms_vec)

# Display results
for msg, pred in zip(new_sms, predictions):
    label = "Spam" if pred == 1 else "Ham"
    print(f"Message: {msg}")
    print(f"Prediction: {label}\n")

Message: Free entry in weekly competition to win cash prize
Prediction: Spam

Message: I'll call you later when I get home
Prediction: Ham



In [61]:
predict_sms(
    "Free entry in weekly competition to win cash prize",
    gb_model,
    tfidf.fit(X_train),
    dense=True
)

'Spam'

# 3. Stacking
- Logistic + SVM + RF ==> LR
- CNN1 + CNN2 ==> MLP


## 3.1. Logistic + SVM + RF ==> LR
### Base learners:
+ Logistic Regression
+ Linear SVM
+ Random Forest
### Meta learner:
+ Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

In [63]:
base_estimators = [
    ("lr", LogisticRegression(
        max_iter=5000,
        class_weight="balanced"
    )),
    ("svm", LinearSVC(
        C=1.0,
        class_weight="balanced"
    )),
    ("rf", RandomForestClassifier(
        n_estimators=200,
        n_jobs=-1,
        random_state=42,
        class_weight="balanced"
    ))
]


In [64]:
stacking_model = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(
        max_iter=5000,
        class_weight="balanced"
    ),
    cv=5,
    n_jobs=-1
)

# Train stacking model
stacking_model.fit(X_train_vec, y_train)


StackingClassifier(cv=5,
                   estimators=[('lr',
                                LogisticRegression(class_weight='balanced',
                                                   max_iter=5000)),
                               ('svm', LinearSVC(class_weight='balanced')),
                               ('rf',
                                RandomForestClassifier(class_weight='balanced',
                                                       n_estimators=200,
                                                       n_jobs=-1,
                                                       random_state=42))],
                   final_estimator=LogisticRegression(class_weight='balanced',
                                                      max_iter=5000),
                   n_jobs=-1)

In [65]:
y_pred = stacking_model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Ham", "Spam"]))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9713004484304932

Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      0.98      0.98       966
        Spam       0.87      0.93      0.90       149

    accuracy                           0.97      1115
   macro avg       0.93      0.95      0.94      1115
weighted avg       0.97      0.97      0.97      1115


Confusion Matrix:
[[945  21]
 [ 11 138]]


In [67]:
joblib.dump(stacking_model, r"E:\Desktop\02VC_Group_09\spamSMS\stacking_lr_svm_rf.joblib")
# joblib.dump(tfidf, r"E:\Desktop\02VC_Group_09\spamSMS\tfidf_vectorizer.joblib")

['E:\\Desktop\\02VC_Group_09\\spamSMS\\stacking_lr_svm_rf.joblib']

In [68]:
predict_sms(
    "Free entry in weekly competition to win cash prize",
    stacking_model,
    tfidf
)

'Spam'

## 3.2. CNN1 + CNN2 ==> MLP

In [71]:
! pip install tensorflow

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 1.0/331.9 MB 2.0 MB/s eta 0:02:45
   ---------------------------------------- 1.8/331.9 MB 2.6 MB/s eta 0:02:09
   ---------------------------------------- 2.4/331.9 MB 2.9 MB/s eta 0:01:56
   ---------------------------------------- 2.9/331.9 MB 2.7 MB/s eta 0:02:04
   -------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.2 which is incompatible.


In [72]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D,
    Dense, Dropout, Concatenate
)
from tensorflow.keras.optimizers import Adam

C:\Users\n33r9\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [73]:
# Vocabulary and sequence settings
MAX_VOCAB = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post")

print("Train shape:", X_train_pad.shape)
print("Test shape:", X_test_pad.shape)


Train shape: (4457, 100)
Test shape: (1115, 100)


In [74]:
# shared embedding layer
EMBED_DIM = 128

input_text = Input(shape=(MAX_LEN,), name="input_text")

embedding = Embedding(
    input_dim=MAX_VOCAB,
    output_dim=EMBED_DIM,
    input_length=MAX_LEN
)(input_text)


C:\Users\n33r9\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [75]:
cnn1 = Conv1D(
    filters=128,
    kernel_size=3,
    activation="relu"
)(embedding)

cnn1 = GlobalMaxPooling1D()(cnn1)


In [76]:
cnn2 = Conv1D(
    filters=128,
    kernel_size=5,
    activation="relu"
)(embedding)

cnn2 = GlobalMaxPooling1D()(cnn2)


In [77]:
# Concatenate CNN outputs
merged = Concatenate()([cnn1, cnn2])

# Meta-classifier (MLP)
x = Dense(128, activation="relu")(merged)
x = Dropout(0.5)(x)
x = Dense(64, activation="relu")(x)

output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=input_text, outputs=output)


In [78]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_text (InputLayer)       │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 100, 128)          │       1,280,000 │ input_text[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 98, 128)           │          49,280 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, 96, 128)           │          82,048 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d          │ (None, 128)               │               0 │ conv1d[0][0]               │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d_1        │ (None, 128)               │               0 │ conv1d_1[0][0]             │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 256)               │               0 │ global_max_pooling1d[0][0… │
│                               │                           │                 │ global_max_pooling1d_1[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │          32,896 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           8,256 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 1)                 │              65 │ dense_1[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,452,545 (5.54 MB)

 Trainable params: 1,452,545 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [79]:
# train model
history = model.fit(
    X_train_pad,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9018 - loss: 0.2735 - val_accuracy: 0.9776 - val_loss: 0.0817
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9920 - loss: 0.0346 - val_accuracy: 0.9821 - val_loss: 0.0600
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.9978 - loss: 0.0086 - val_accuracy: 0.9865 - val_loss: 0.0714
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9998 - loss: 0.0016 - val_accuracy: 0.9910 - val_loss: 0.0907
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.9998 - loss: 0.0010 - val_accuracy: 0.9910 - val_loss: 0.0974
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 1.0000 - loss: 2.7776e-04 - val_accuracy: 0.9910 - val_loss: 0.1024
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 1.0000 - loss: 2.6530e-04 - val_accuracy: 0.9910 - val_loss: 0.1164
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 1.0000 - loss: 2.0736e-0

In [80]:
# Predict probabilities
y_pred_prob = model.predict(X_test_pad)

# Convert to class labels
y_pred = (y_pred_prob > 0.5).astype(int).ravel()

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Ham", "Spam"]))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      0.99      0.99       966
        Spam       0.96      0.91      0.93       149

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Confusion Matrix:
[[960   6]
 [ 14 135]]


In [81]:
model.save(r"E:\Desktop\02VC_Group_09\spamSMS\cnn_sms_model.h5")

In [82]:
def predict_sms_cnn(message):
    seq = tokenizer.texts_to_sequences([message])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding="post")
    prob = model.predict(pad)[0][0]
    label = "Spam" if prob > 0.5 else "Ham"
    return label, prob

# Example
predict_sms_cnn("Congratulations! You have won a free prize. Call now")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


('Spam', 0.9999991)

# Comparison table

In [87]:
# compare 5 models
# Bagging (Random Forest, SVM), AdaBoost, Gradient Boosting, Stacking (LR, SVM, RF), CNN
results = pd.DataFrame({
    "Model": [
        "Random Forest (Bagging)",
        "SVM (Bagging)",
        "AdaBoost",
        "Gradient Boosting",
        "Stacking (LR, SVM, RF)",
        "CNN1+CNN2=>MLP"
    ],
    "Accuracy": [
        accuracy_score(y_test, rf_model.predict(X_test_vec)),
        accuracy_score(y_test, bagging_svm.predict(X_test_vec)),
        accuracy_score(y_test, adaboost_model.predict(X_test_vec)),
        accuracy_score(y_test, gb_model.predict(X_test_vec.toarray())),
        accuracy_score(y_test, stacking_model.predict(X_test_vec)),
        accuracy_score(y_test, (model.predict(X_test_pad) > 0.5).astype(int).ravel())
    ],
    "Precision": [
        classification_report(y_test, rf_model.predict(X_test_vec), output_dict=True)['1']['precision'],
        classification_report(y_test, bagging_svm.predict(X_test_vec), output_dict=True)['1']['precision'],
        classification_report(y_test, adaboost_model.predict(X_test_vec), output_dict=True)['1']['precision'],
        classification_report(y_test, gb_model.predict(X_test_vec.toarray()), output_dict=True)['1']['precision'],
        classification_report(y_test, stacking_model.predict(X_test_vec), output_dict=True)['1']['precision'],
        classification_report(y_test, (model.predict(X_test_pad) > 0.5).astype(int).ravel(), output_dict=True)['1']['precision']
    ],
    "Recall": [
        classification_report(y_test, rf_model.predict(X_test_vec), output_dict=True)['1']['recall'],
        classification_report(y_test, bagging_svm.predict(X_test_vec), output_dict=True)['1']['recall'],
        classification_report(y_test, adaboost_model.predict(X_test_vec), output_dict=True)['1']['recall'],
        classification_report(y_test, gb_model.predict(X_test_vec.toarray()), output_dict=True)['1']['recall'],
        classification_report(y_test, stacking_model.predict(X_test_vec), output_dict=True)['1']['recall'],
        classification_report(y_test, (model.predict(X_test_pad) > 0.5).astype(int).ravel(), output_dict=True)['1']['recall']
    ],
    "F1-Score": [
        classification_report(y_test, rf_model.predict(X_test_vec), output_dict=True)['1']['f1-score'],
        classification_report(y_test, bagging_svm.predict(X_test_vec), output_dict=True)['1']['f1-score'],
        classification_report(y_test, adaboost_model.predict(X_test_vec), output_dict=True)['1']['f1-score'],
        classification_report(y_test, gb_model.predict(X_test_vec.toarray()), output_dict=True)['1']['f1-score'],
        classification_report(y_test, stacking_model.predict(X_test_vec), output_dict=True)['1']['f1-score'],
        classification_report(y_test, (model.predict(X_test_pad) > 0.5).astype(int).ravel(), output_dict=True)['1']['f1-score']
    ]          
})
results

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


,Model,Accuracy,Precision,Recall,F1-Score
0,Random Forest (Bagging),0.976682,1.000000,0.825503,0.904412
1,SVM (Bagging),0.982960,0.964286,0.906040,0.934256
2,AdaBoost,0.974888,0.948148,0.859060,0.901408
3,Gradient Boosting,0.969507,0.975207,0.791946,0.874074
4,"Stacking (LR, SVM, RF)",0.971300,0.867925,0.926174,0.896104
5,CNN1+CNN2=>MLP,0.982063,0.957447,0.906040,0.931034
